In [1]:
import tensorflow as tf
keras = tf.keras

2023-06-13 03:48:40.053173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 03:48:40.943696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model=keras.models.load_model("/tmp/open_spiel_PVPNet_test_mlp.pb")

2023-06-11 00:30:47.411439: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-11 00:30:47.483468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-11 00:30:47.483677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [9]:
batch_size=30
environment_shape=(20,20,2)
state_shape=(1,)
E=tf.zeros([batch_size,*environment_shape])
S=tf.zeros([batch_size,*state_shape])
model.predict({"inputs_0":E,"inputs_1":S})

ValueError: in user code:

    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non

In [10]:
model.

In [247]:
n=5
G=tf.cast(tf.random.uniform([n,n]) > 0.5,dtype=tf.float32)
W=tf.random.uniform([n,n]) * G
x=tf.random.uniform([n,1])
x=tf.stack([x,x])
G=tf.stack([G,G])
W=tf.stack([W,W])

import importlib
import mpg.gnn.gcn as gcn
import mpg.gnn.aggregator as agg
import mpg.gnn.sgcn as sgcn 
importlib.reload(gcn)
importlib.reload(agg)
importlib.reload(sgcn)
WGCN=gcn.WGCN

In [248]:
layer=gcn.GrpahConv(5,loops=False,normalization="unit")
layer2=WGCN(5,loops=False,normalization="unit")
Y=layer((G,W,x))
Z=layer2((G,W,Y))


In [249]:
Y

<tf.Tensor: shape=(2, 1, 5), dtype=float32, numpy=
array([[[-0.92741394, -0.92741394, -0.92741394, -0.92741394,
         -0.92741394]],

       [[-0.92741394, -0.92741394, -0.92741394, -0.92741394,
         -0.92741394]]], dtype=float32)>

[<tf.Variable 'TAGConv/kernel:0' shape=(1, 5, 2) dtype=float32, numpy=
 array([[[-0.04742214, -0.14084202],
         [-0.41123575,  0.13933949],
         [ 0.72605807,  0.03667752],
         [-1.2124313 ,  0.43771482],
         [-0.48633334,  0.4591117 ]]], dtype=float32)>]

In [163]:
aggregator1=agg.SumAggregation(loops=True,with_weights=False,normalization="unit")
def f(A,x):
    matrices=A
    matrices=matrices + tf.eye(num_rows=matrices.shape[-1],batch_shape=matrices.shape[:-2])
    degrees=tf.reduce_sum(matrices,axis=-1)
    D=tf.sqrt(tf.linalg.diag(degrees))
    return tf.linalg.matmul(tf.linalg.matmul(tf.linalg.pinv(D),tf.linalg.matmul(matrices,tf.linalg.pinv(D))),x)

data=tf.ones([2,5,1])

In [168]:
f(G,x)

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.4892034 ],
        [0.52406955]],

       [[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.4892034 ],
        [0.52406955]]], dtype=float32)>

In [169]:
aggregator1((G,x))

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.48920345],
        [0.52406955]],

       [[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.48920345],
        [0.52406955]]], dtype=float32)>

In [170]:
import numpy as np
P=np.zeros([2,5,1])
degree = tf.reduce_sum(G,axis=-1) +1 
for b in range(2):
    
    for i in range(5):
        P[b,i,0] = x[b,i,0] / degree[b,i]
        for j in range(5):
            P[b,i,0] += G[b,i,j]*x[b,j,0] / np.sqrt(degree[b,i] * degree[b,j])
P

array([[[0.2452044 ],
        [0.65653056],
        [0.64284056],
        [0.48920345],
        [0.52406955]],

       [[0.2452044 ],
        [0.65653056],
        [0.64284056],
        [0.48920345],
        [0.52406955]]])

In [167]:
matrices=G
matrices=matrices + tf.eye(num_rows=matrices.shape[-1],batch_shape=matrices.shape[:-2])
degrees=tf.reduce_sum(matrices,axis=-1,keepdims=True)
tf.linalg.matmul(matrices, data / tf.math.sqrt(degrees))/tf.math.sqrt(degrees)

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.90824825],
        [1.        ],
        [1.1422286 ],
        [1.0749149 ],
        [1.        ]],

       [[0.90824825],
        [1.        ],
        [1.1422286 ],
        [1.0749149 ],
        [1.        ]]], dtype=float32)>

In [154]:
degrees.shape

TensorShape([2, 5])

In [129]:
import tensorflow_gnn as tfgnn

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [127]:
Z[10,20]

<tf.Tensor: shape=(), dtype=float32, numpy=0.15104508>

In [131]:
!protoc --version

libprotoc 3.19.6
